<a href="https://colab.research.google.com/github/sjpark0605/NLP-FYP/blob/main/NER_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%capture
!pip install datasets

In [24]:
# Imports for Data Processing
import glob
import csv
import pandas as pd
from datasets import Dataset, ClassLabel, Sequence, DatasetDict

In [25]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive/')

project_dir = '/content/drive/MyDrive/COMP0029/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
corpus_list = ['r-100', 'r-200', 'r-300']  
for target_corpus in corpus_list:
  recipe_files = []
  
  if target_corpus != 'r-300':
    recipe_files += glob.glob(project_dir + target_corpus + '/*.list')
  else:
    recipe_files += glob.glob(project_dir + 'r-100/*.list')
    recipe_files += glob.glob(project_dir + 'r-200/*.list')

  recipe_ner_data_csv = open(project_dir + target_corpus + '-recipe-ner-data.csv', "w", encoding="utf8")
  writer = csv.writer(recipe_ner_data_csv)

  header = ['Sentence Number', 'Word', 'POS', 'Label']
  writer.writerow(header)

  sentence_no = 1

  for file in recipe_files:
    recipe_data = open(file, "r", encoding="utf8")
    lines = recipe_data.readlines()

    for line in lines:
      items = line.split(" ")
      word = items[3]
      pos = items[4]
      label = items[5].replace("\n", "")

      row = ["Sentence_" + str(sentence_no), word, pos, label]
      writer.writerow(row)

      if pos == '.':
        sentence_no += 1
      
    recipe_data.close()

  recipe_ner_data_csv.close()

In [27]:
df = pd.read_csv(project_dir + 'r-300-recipe-ner-data.csv')
pos_list = df['POS'].unique()
label_list = sorted(df['Label'].unique())

label_list.remove('O')
label_list.append('O')

grouped = df.groupby('Sentence Number').agg({'Word': list, 'POS': list, 'Label': list}).reset_index()
grouped.drop('Sentence Number', axis=1, inplace=True) 
grouped.rename(columns={"Word": "tokens", "POS": "pos", "Label": "ner_tags"}, inplace=True)

dataset = Dataset.from_pandas(grouped)
dataset = dataset.cast_column("pos", Sequence(ClassLabel(names=list(pos_list))))
dataset = dataset.cast_column("ner_tags", Sequence(ClassLabel(names=list(label_list))))

Casting the dataset:   0%|          | 0/2752 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2752 [00:00<?, ? examples/s]

In [28]:
dataset = dataset.shuffle()
split_dataset = dataset.train_test_split(test_size=0.2)

corpus_datasets = DatasetDict({
    "train": split_dataset["train"],
    "valid": split_dataset["test"]
})

corpus_datasets.save_to_disk(project_dir + 'datasets/english-recipe-ner')

Flattening the indices:   0%|          | 0/2201 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2201 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/551 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/551 [00:00<?, ? examples/s]